In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import mxnet as mx
from mxnet import gluon

# !pip install gluonts



**gluon in kaggle:**  
https://www.kaggle.com/steverab/m5-forecasting-competition-gluonts-template  
https://www.kaggle.com/aldente0630/demand-forecasting-with-deepar-and-deepvar

**gluon multiforecasting::**  
https://github.com/awslabs/gluon-ts/issues/1095  
https://githubmemory.com/repo/awslabs/gluon-ts/issues/1095   

**crypto forecasting:**   
https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition/notebook  
https://github.com/pedrolarben/TimeSeriesForecasting-DeepLearning  
https://www.kaggle.com/negstek/to-the-moon-g-research-crypto-forecasting-eda/edit  
https://www.kaggle.com/yamqwe/crypto-prediction-catboost-regressor

**lint et méthodo:**  
https://blog.reviewnb.com/development-with-jupyter/   
https://towardsdatascience.com/4-tools-for-reproducible-jupyter-notebooks-d7423721bd04  

This first test is based on [this notebook](https://www.kaggle.com/yamqwe/crypto-prediction-catboost-regressor/notebook)

In [ ]:
import numpy as np
import pandas as pd
import gresearch_crypto 
from catboost import Pool, CatBoostClassifier, cv, CatBoostRegressor

In [ ]:
TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

In [ ]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

In [ ]:
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.drop_duplicates(inplace=True)

In [ ]:
def upper_shadow(df): return df['High'] - np.maximum(df['Close'], df['Open'])
def lower_shadow(df): return np.minimum(df['Close'], df['Open']) - df['Low']

# Credit: https://www.kaggle.com/swaralipibose/64-new-features-with-autoencoders/code
def get_features(df, row = False):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['upper_Shadow'] = upper_shadow(df_feat)
    df_feat['lower_Shadow'] = lower_shadow(df_feat)
    df_feat["high_div_low"] = df_feat["High"] / df_feat["Low"]
    df_feat['trade'] = df_feat['Close'] - df_feat['Open']
    df_feat['gtrade'] = df_feat['trade'] / df_feat['Count']
    df_feat['shadow1'] = df_feat['trade'] / df_feat['Volume']
    df_feat['shadow3'] = df_feat['upper_Shadow'] / df_feat['Volume']
    df_feat['shadow5'] = df_feat['lower_Shadow'] / df_feat['Volume']
    df_feat['diff1'] = df_feat['Volume'] - df_feat['Count']
    df_feat['mean1'] = (df_feat['shadow5'] + df_feat['shadow3']) / 2
    df_feat['mean2'] = (df_feat['shadow1'] + df_feat['Volume']) / 2
    df_feat['mean3'] = (df_feat['trade'] + df_feat['gtrade']) / 2
    df_feat['mean4'] = (df_feat['diff1'] + df_feat['upper_Shadow']) / 2
    df_feat['mean5'] = (df_feat['diff1'] + df_feat['lower_Shadow']) / 2
    return df_feat
def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=10, random_seed=42, verbose = 0)
    model.fit(X, y)

    return X, y, model

In [ ]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(df_train, asset_id)    
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

In [ ]:
# Check the model interface
x = get_features(df_train.iloc[1],row=True)
y_pred = models[0].predict([x])
y_pred[0]

In [ ]:
# import pdb; pdb.set_trace()

In [ ]:
env = gresearch_crypto.make_env()


In [ ]:
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row,row=True)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)

    # Send submissions
    env.predict(df_pred)